In [119]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score, classification_report, log_loss, make_scorer
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_log_error
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import date
import datetime

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [120]:
df = pd.read_csv('data/train.csv')

In [121]:
# df[['genres','genres1','genres2','genres3','genres4','genres5','genres6','genres7']] = df['genres'].str.split(',', expand=True).drop(columns=[0,2,4,6,8,10,12])

In [122]:
df['genres'].str.split(',', expand=True).drop(columns=[0,2,4,6,8,10,12])


,1,3,5,7,9,11,13
0,'name': 'Comedy'}],None,None,None,None,None,None
1,'name': 'Comedy'},'name': 'Drama'},'name': 'Family'},'name': 'Romance'}],None,None,None
2,'name': 'Drama'}],None,None,None,None,None,None
3,'name': 'Thriller'},'name': 'Drama'}],None,None,None,None,None
4,'name': 'Action'},'name': 'Thriller'}],None,None,None,None,None
5,'name': 'Animation'},'name': 'Adventure'},'name': 'Family'}],None,None,None,None
6,'name': 'Horror'},'name': 'Thriller'}],None,None,None,None,None
7,'name': 'Documentary'}],None,None,None,None,None,None
8,'name': 'Action'},'name': 'Comedy'},'name': 'Music'},'name': 'Family'},'name': 'Adventure'}],None,None
9,'name': 'Comedy'},'name': 'Music'}],None,None,None,None,None


In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 23 columns):
id                       3000 non-null int64
belongs_to_collection    604 non-null object
budget                   3000 non-null int64
genres                   2993 non-null object
homepage                 946 non-null object
imdb_id                  3000 non-null object
original_language        3000 non-null object
original_title           3000 non-null object
overview                 2992 non-null object
popularity               3000 non-null float64
poster_path              2999 non-null object
production_companies     2844 non-null object
production_countries     2945 non-null object
release_date             3000 non-null object
runtime                  2998 non-null float64
spoken_languages         2980 non-null object
status                   3000 non-null object
tagline                  2403 non-null object
title                    3000 non-null object
Keywords             

In [124]:
df['runtime'].fillna((df['runtime'].mean()), inplace=True)
df['release_date'] = pd.to_datetime(df['release_date'])
df['release_date'] = datetime.date.today().year
df = pd.get_dummies(df, prefix=['Is'], columns=['original_language'])

In [125]:
df.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_title,overview,popularity,poster_path,...,Is_ro,Is_ru,Is_sr,Is_sv,Is_ta,Is_te,Is_tr,Is_ur,Is_vi,Is_zh
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,/tQtWuwvMf0hCc2QR2tkolwl7c3c.jpg,...,0,0,0,0,0,0,0,0,0,0
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,/w9Z7A0GHEhIp7etpj0vyKOeU1Wx.jpg,...,0,0,0,0,0,0,0,0,0,0
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,/lIv1QinFqz4dlp5U4lQ6HaiskOZ.jpg,...,0,0,0,0,0,0,0,0,0,0
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,/aTXRaPrWSinhcmCrcfJK17urp3F.jpg,...,0,0,0,0,0,0,0,0,0,0
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,마린보이,Marine Boy is the story of a former national s...,1.148070,/m22s7zvkVFDU9ir56PiiqIEWFdT.jpg,...,0,0,0,0,0,0,0,0,0,0


In [126]:
# for i in range(len(temp)):
#     print(temp[i]['name'])


In [127]:
y = df.revenue
X = df[['id','budget', 'popularity','runtime','release_date','Is_en']]
#'production_countries', 

In [128]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 6 columns):
id              3000 non-null int64
budget          3000 non-null int64
popularity      3000 non-null float64
runtime         3000 non-null float64
release_date    3000 non-null int64
Is_en           3000 non-null uint8
dtypes: float64(2), int64(3), uint8(1)
memory usage: 120.2 KB


In [129]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [130]:
rf = RandomForestRegressor(n_estimators=100,
                            n_jobs=-1,
                            random_state=1)

In [131]:
random_forest_grid = {'max_depth': [10],
                      'max_features': ['sqrt', 'log2', None],
                      'min_samples_split': [2,4],
                      'min_samples_leaf': [1, 2, 4],
                      'bootstrap': [True, False],
                      'n_estimators': [10,20,40,80,160],
                      'random_state': [1]}

rf_gridsearch = GridSearchCV(RandomForestRegressor(),
                             random_forest_grid,
                             n_jobs=-1,
                             verbose=True,
                             scoring='neg_mean_squared_error')

rf_gridsearch.fit(X_train, y_train)

print( "best parameters:", rf_gridsearch.best_params_ )

best_rf_model = rf_gridsearch.best_estimator_

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 180 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Done 232 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:   15.3s finished


best parameters: {'bootstrap': True, 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 160, 'random_state': 1}


In [132]:
best_rf_model

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features=None, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=4, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=160, n_jobs=None,
           oob_score=False, random_state=1, verbose=0, warm_start=False)

In [133]:
best_rf_model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features=None, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=4, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=160, n_jobs=None,
           oob_score=False, random_state=1, verbose=0, warm_start=False)

In [134]:
y_hat = best_rf_model.predict(X_test)

In [137]:
mean_squared_error(y_test,y_hat)

5424881961030588.0

In [138]:
mean_squared_log_error(y_test,y_hat)

6.065668107121707